## train the LM 

In [ ]:
import sys

In [ ]:
!{sys.executable} -m pip install https://github.com/kpu/kenlm/archive/master.zip 
!{sys.executable} -m pip install pyctcdecode==0.3.0
!{sys.executable} -m pip install datasets==2.0.0
!{sys.executable} -m pip install transformers==4.18.0

In [ ]:
!{sys.executable} -m pip install ipywidgets

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!sudo apt install build-essential cmake libboost-system-dev libboost-thread-dev libboost-program-options-dev libboost-test-dev libeigen3-dev zlib1g-dev libbz2-dev liblzma-dev


In [ ]:
!wget -O - https://kheafield.com/code/kenlm.tar.gz | tar xz


In [ ]:
!mkdir kenlm/build && cd kenlm/build && cmake .. && make -j2

In [ ]:
!ls kenlm/build/bin

In [ ]:
target_lang="sv"  # change to your target lang


In [ ]:
from datasets import load_dataset

username = "hf-test"  # change to your username

dataset = load_dataset(f"{username}/{target_lang}_corpora_parliament_processed", split="train")

with open("text.txt", "w") as file:
  file.write(" ".join(dataset["text"]))


In [ ]:
!kenlm/build/bin/lmplz -o 5 <"text.txt" > "5gram.arpa"


In [ ]:
!head -20 5gram.arpa


In [ ]:
with open("5gram.arpa", "r") as read_file, open("5gram_correct.arpa", "w") as write_file:
  has_added_eos = False
  for line in read_file:
    if not has_added_eos and "ngram 1=" in line:
      count=line.strip().split("=")[-1]
      write_file.write(line.replace(f"{count}", f"{int(count)+1}"))
    elif not has_added_eos and "<s>" in line:
      write_file.write(line)
      write_file.write(line.replace("<s>", "</s>"))
      has_added_eos = True
    else:
      write_file.write(line)

In [ ]:
!head -20 5gram_correct.arpa

In [ ]:
from transformers import AutoProcessor

processor = AutoProcessor.from_pretrained("/home/ubuntu/wav2vec2-xlsr-xls-r-300m/checkpoint-41600")

In [ ]:
vocab_dict = processor.tokenizer.get_vocab()
sorted_vocab_dict = {k.lower(): v for k, v in sorted(vocab_dict.items(), key=lambda item: item[1])}

In [ ]:
sorted_vocab_dict

In [ ]:
#from pyctcdecode import build_ctcdecoder


#labels = [
#    "s","b","l","u","ç","i","o","f","d","k","ɔ","t","r","w","j","p","x","v","c","z",
#    "h","q","y","m","|","e","n","ε","a","g","[UNK]","[PAD]","<s>","</s>"
#]
    
# prepare decoder and decode logits via shallow fusion
#decoder = build_ctcdecoder(
#    labels,
#    kenlm_model_path="5gram_correct.arpa",  # either .arpa or .bin file
#)

In [ ]:
from pyctcdecode import build_ctcdecoder

decoder = build_ctcdecoder(
    labels=list(sorted_vocab_dict.keys()),
    kenlm_model_path="5gram_correct.arpa",
)

In [ ]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("/home/ubuntu/wav2vec2-xlsr-xls-r-300m/checkpoint-41600", eos_token=None, bos_token=None)

In [ ]:
from transformers import Wav2Vec2ProcessorWithLM

processor_with_lm = Wav2Vec2ProcessorWithLM(
    feature_extractor=processor.feature_extractor,
    tokenizer=processor.tokenizer,
    decoder=decoder
)

In [ ]:
from huggingface_hub import Repository

repo = Repository(local_dir="xlsr-fleurs", clone_from="Ussen/wav2vec2-large-xlsr-lingala-fleurs4")

In [ ]:
processor_with_lm.save_pretrained("xlsr-fleurs300m")

In [ ]:
#use the terminal
!kenlm/build/bin/build_binary xlsr-fleurs300m/language_model/5gram_correct.arpa xlsr-fleurs300m/language_model/5gram.bin

In [ ]:
!rm xlsr-fleurs300m/language_model/5gram_correct.arpa && tree -h xlsr-fleurs300m/


## push to hub

In [ ]:
!git clone https://huggingface.co/Ussen/xlsr-lingala2

In [ ]:
# this code worked with the following torch setup
!{sys.executable} -m pip install torch==1.12.0+cu116 torchvision==0.13.0+cu116 torchaudio==0.12.0 --extra-index-url https://download.pytorch.org/whl/cu116

In [1]:
from transformers import Wav2Vec2ProcessorWithLM, Wav2Vec2ForCTC

processor = Wav2Vec2ProcessorWithLM.from_pretrained("/home/ubuntu/LM/xlsr-fleurs300m")
model = Wav2Vec2ForCTC.from_pretrained("/home/ubuntu/LM/xlsr-fleurs300m")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
from transformers import AutoProcessor

processor1 = AutoProcessor.from_pretrained("/home/ubuntu/wav2vec2-xlsr-xls-r-300m/checkpoint-41600")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## load the test set

In [3]:
import os

In [4]:
import sys

In [5]:
from datasets import load_dataset
fleurs_asr = load_dataset("google/fleurs", "ln_cd") # for Lingala
# to download all data for multi-lingual fine-tuning uncomment following line
# fleurs_asr = load_dataset("google/fleurs", "all")
# see structure
print(fleurs_asr)
# load audio sample on the fly
audio_input = fleurs_asr["train"][0]["audio"] # first decoded audio sample
transcription = fleurs_asr["train"][0]["transcription"] # first transcription
# use `audio_input` and `transcription` to fine-tune your model for ASR
# for analyses see language groups
all_language_groups = fleurs_asr["train"].features["lang_group_id"].names
lang_group_id = fleurs_asr["train"][0]["lang_group_id"]
all_language_groups[lang_group_id]

Reusing dataset fleurs (/home/ubuntu/.cache/huggingface/datasets/google___fleurs/ln_cd/2.0.0/af82dbec419a815084fa63ebd5d5a9f24a6e9acdf9887b9e3b8c6bbd64e0b7ac)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'num_samples', 'path', 'audio', 'transcription', 'raw_transcription', 'gender', 'lang_id', 'language', 'lang_group_id'],
        num_rows: 3350
    })
    validation: Dataset({
        features: ['id', 'num_samples', 'path', 'audio', 'transcription', 'raw_transcription', 'gender', 'lang_id', 'language', 'lang_group_id'],
        num_rows: 209
    })
    test: Dataset({
        features: ['id', 'num_samples', 'path', 'audio', 'transcription', 'raw_transcription', 'gender', 'lang_id', 'language', 'lang_group_id'],
        num_rows: 478
    })
})


'sub_saharan_african_ssa'

In [6]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [7]:
fleurs_asr_train = fleurs_asr['train'].remove_columns(["id", "num_samples", "raw_transcription", "gender", "lang_id", "language", "lang_group_id"])
fleurs_asr_test = fleurs_asr['validation'].remove_columns(["id", "num_samples", "raw_transcription", "gender", "lang_id", "language", "lang_group_id"])

In [8]:
show_random_elements(fleurs_asr_train.remove_columns(["path", "audio"]), num_examples=10)

,transcription
0,ezalaki kati na bitambe ya ntina mingi na likambo ya ntina ya henry louis gates etali makamwisi ya mikili ya afrika
1,mpepo ezalaki kokende na irkoutsk mpe bazalaki kotambwisa yango na basoda bazalaki na kati
2,christopher garcia molobeli ya bapolisi ya los angeles alobaki ete moto ya mobulu oyo bazalaka kolandela bazalaki kosala yango po amikotisaki na ndako ya bato kasi te mpona mobulu makasi
3,ba casques bleus ya onu oyo bayaki na haïti nsima ya koningana ya mabele ya mobu 2010 bazopesa bango foti ete bango nde bazopanza maladi oyo ebendaka pembeni ya esika bafandaka
4,mabele eninganaki na mariana 07h19 na ntongo na ngonga ya esika wana 09h19 ya pokwa gmt na mokolo ya mitano
5,mayele ya kotonga bandako architecture etaleli kosala baplan ya bandako mpe kotonga yango. mbala mingi architecture ya esika moko ebendaka mpenza likebi ya bato oyo bakendaka kotala bamboka ya bapaya
6,mbala mingi basengaka mbongo mpona komikomisa na manaka wana ya mateya
7,équateur ezosenga ete bato ya cuba bazwa mokanda ya libiangi liboso bakota na équateur na nzela ya libanda ya mpepo ya mikili mingi to na bangambo mosusu ya kokotela na mikili
8,mobeko ya france ebongwani. misala na ye ya mabe ebanda mibu 15 eleki ntango asanganaki na botomboki ya france na bitumba ya mibale ya mokili mobimba
9,équateur ezosenga ete bato ya cuba bazwa mokanda ya libiangi liboso bakota na équateur na nzela ya libanda ya mpepo ya mikili mingi to na bangambo mosusu ya kokotela na mikili


In [9]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\[\]\¥\+\ł\£\—\½\~\$\&\¾\²\°\/\–]'

def remove_special_characters(batch):
    batch["transcription"] = re.sub(chars_to_ignore_regex, '', batch["transcription"]).lower() + " "
    return batch

In [10]:
fleurs_asr_train = fleurs_asr_train.map(remove_special_characters)
fleurs_asr_test = fleurs_asr_test.map(remove_special_characters)

Parameter 'function'=<function remove_special_characters at 0x7f39bb35a9e0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/3350 [00:00<?, ?ex/s]

  0%|          | 0/209 [00:00<?, ?ex/s]

In [11]:
show_random_elements(fleurs_asr_test.remove_columns(["path","audio"]))

,transcription
0,ntango bitumba esilaki nsima ya komema bato bazokaki na lopitalo pene ya bato mosusu 40 ya boloko oyo batikalaki batikalaki na lopango pe baboyaki kozonga na baselile na bango na boloko
1,okoki komona bapiramide na molili mpe okoki komona yango na kimia liboso spectacle ebanda
2,salela esika makambo na lokumu limemia pe botosi oyo ebongi na yango kosala maseki te mpona holocauste to banazi
3,eleko ya mpio makasi ekoki kozala na malili ya lokuta ba températures ekitaka mpenza te na se ya libungutulu kasi mipepe pe malili esanganaka mpona kosala neti malili eza mingi koleka ndenge termometele ezolakisa
4,mabota ebele ya baebele bazalaki kolekisa bomoi na bango na libanda
5,liboso lisanga moko esala makambo ya sika bakambi basengeli kokolisa mayele ya kosala makambo ya sika mpe kokabola boyebi pe mayele ya kobongisa makambo
6,bato ya siansi balobi oyo kopanzana wana ezalaki ya makasi mpenza
7,mbongwana to bosengi nyonso esengeli kosalema na nzela ya kompani ya mobembo kasi te na nzela ya hotel
8,molobeli ya bush gordon johndroe abengaki komipesa ya corée ya nordi etambe monene mpona kokokisa mokano ya kolongola mandoki ya nikleyere na esanga ya corée
9,bandakisa ya misala oyo esalemaka ezali kosala bokila koloba mbisi kokanga bafoto kotala bandeke pe kotala baparke mpe koyekola basango etali ndenge mabele ezali


In [12]:
def extract_all_chars(batch):
  all_text = " ".join(batch["transcription"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [13]:
vocab_train = fleurs_asr_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=fleurs_asr_train.column_names)
vocab_test = fleurs_asr_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=fleurs_asr_test.column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [14]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

In [15]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'w': 0,
 's': 1,
 'ã': 2,
 'j': 3,
 'à': 4,
 'á': 5,
 'a': 6,
 '7': 7,
 ' ': 8,
 'ü': 9,
 'y': 10,
 't': 11,
 'f': 12,
 'ó': 13,
 'â': 14,
 'i': 15,
 'b': 16,
 '8': 17,
 'ú': 18,
 '9': 19,
 'œ': 20,
 'x': 21,
 '4': 22,
 'ö': 23,
 'k': 24,
 'o': 25,
 'h': 26,
 'n': 27,
 'õ': 28,
 'u': 29,
 'q': 30,
 'z': 31,
 'í': 32,
 "'": 33,
 'c': 34,
 'r': 35,
 'e': 36,
 '1': 37,
 '5': 38,
 '0': 39,
 'é': 40,
 'è': 41,
 '6': 42,
 'd': 43,
 'g': 44,
 'ñ': 45,
 '2': 46,
 '3': 47,
 'p': 48,
 'ï': 49,
 'ç': 50,
 'm': 51,
 'î': 52,
 'l': 53,
 'v': 54}

In [16]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [17]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

57

In [18]:
fleurs_asr_train[0]["path"]

'/home/ubuntu/.cache/huggingface/datasets/downloads/extracted/b974d17bcda09ca396fb4a9c4d22954318c721e035c18f8e3ed95d9e5327f1c7/10003306233985436987.wav'

In [19]:
fleurs_asr_train[0]["audio"]

{'path': 'train/10003306233985436987.wav',
 'array': array([ 0.        ,  0.        ,  0.        , ..., -0.00193626,
        -0.00319463, -0.00193232]),
 'sampling_rate': 16000}

In [20]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["transcription"]).input_ids
    return batch

In [21]:
#fleurs_asr_train = fleurs_asr_train.map(prepare_dataset, remove_columns=fleurs_asr_train.column_names, batch_size=2, num_proc=1, batched=False)
#fleurs_asr_test = fleurs_asr_test.map(prepare_dataset, remove_columns=fleurs_asr_test.column_names, batch_size=2, num_proc=1, batched=False)
fleurs_asr_train = fleurs_asr_train.map(prepare_dataset, remove_columns=fleurs_asr_train.column_names, num_proc=2)
fleurs_asr_test = fleurs_asr_test.map(prepare_dataset, remove_columns=fleurs_asr_test.column_names, num_proc=2)

#1:   0%|          | 0/1675 [00:00<?, ?ex/s]

#0:   0%|          | 0/1675 [00:00<?, ?ex/s]

#0:   0%|          | 0/105 [00:00<?, ?ex/s]

#1:   0%|          | 0/104 [00:00<?, ?ex/s]

In [ ]:
#show_random_elements(lingala_train.remove_columns(["path"]))

In [ ]:
#def extract_all_chars(batch):
#  all_text = " ".join(batch["sentence"])
#  vocab = list(set(all_text))
#  return {"vocab": [vocab], "all_text": [all_text]}

In [ ]:
#vocab_train = lingala_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=lingala_train.column_names)
#vocab_test = lingala_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=lingala_test.column_names)
#vocab_valid = lingala_valid.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=lingala_valid.column_names)

In [ ]:
#vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]) | set(vocab_valid["vocab"][0]))

In [ ]:
#vocab_dict = {v: k for k, v in enumerate(vocab_list)}
#vocab_dict

In [ ]:
#vocab_dict["|"] = vocab_dict[" "]
#del vocab_dict[" "]

In [ ]:
#vocab_dict["[UNK]"] = len(vocab_dict)
#vocab_dict["[PAD]"] = len(vocab_dict)
#len(vocab_dict)

In [ ]:
#from transformers import Wav2Vec2ProcessorWithLM

#processor = Wav2Vec2ProcessorWithLM.from_pretrained("/content/xlsr-lingala1")
#model = Wav2Vec2ForCTC.from_pretrained("/content/xlsr-lingala1")

In [ ]:
#import torchaudio

#def speech_file_to_array_fn(batch):
    #speech_array, sampling_rate = torchaudio.load(batch["path"])
    #batch["speech"] = speech_array[0].numpy()
    #batch["sampling_rate"] = sampling_rate
    #batch["target_text"] = batch["sentence"]
    #return batch

In [ ]:
#lingala_train = lingala_train.map(speech_file_to_array_fn, remove_columns=lingala_train.column_names)
#lingala_test = lingala_test.map(speech_file_to_array_fn, remove_columns=lingala_test.column_names)
#lingala_valid = lingala_valid.map(speech_file_to_array_fn, remove_columns=lingala_valid.column_names)

In [ ]:
#def prepare_dataset(batch):
    # check that all files have the correct sampling rate
   # assert (
    #    len(set(batch["sampling_rate"])) == 1
    #), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    #batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values
    
    #with processor.as_target_processor():
       # batch["labels"] = processor(batch["target_text"]).input_ids
    #return batch

In [ ]:
#lingala_train = lingala_train.map(prepare_dataset, remove_columns=lingala_train.column_names, batch_size=4, num_proc=2, batched=True)
#lingala_test = lingala_test.map(prepare_dataset, remove_columns=lingala_test.column_names, batch_size=4, num_proc=2, batched=True)
#lingala_valid = lingala_valid.map(prepare_dataset, remove_columns=lingala_valid.column_names, batch_size=4, num_proc=2, batched=True)

## test a single file

In [22]:
import librosa
import torchaudio

In [23]:
# load the audio data (use your own wav file here!)
input_audio, sr = librosa.load('/home/ubuntu/LM/LingalaAudio/pyLingala-master/data/test/wav/deborah/220718-132354_lin_9f7_elicit_11.wav', sr=16000)

In [24]:
# tokenize
input_values = processor(input_audio, return_tensors="pt")

It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


In [25]:
# retrieve logits
import torch

with torch.no_grad():
  logits = model(**input_values).logits

In [26]:
# decode using n-gram
transcription = processor.batch_decode(logits.numpy()).text


In [27]:
transcription[0].lower()

'ekomeli ngaboni'

In [ ]:
import IPython.display as ipd

audio_sample = "/home/ubuntu/LM/LingalaAudio/pyLingala-master/data/test/wav/deborah/220718-132354_lin_9f7_elicit_11.wav"
ipd.Audio(data=audio_sample, autoplay=True, rate=audio_sample)

import torch
import torchaudio
import soundfile

waveform, sampling_rate = soundfile.read( audio_sample )

features = processor(waveform, sampling_rate=16000, return_tensors="pt")

#input_values = features.input_values
#attention_mask = features.attention_mask

with torch.no_grad():
  logits = model(**features).logits

transcription = processor.batch_decode(logits.numpy()).text
transcription[0].lower()

In [28]:
import io
import os
import math
import torch
import warnings
from tqdm.notebook import tqdm


In [ ]:
#input_dict = processor(lingala_test["input_values"][50],sampling_rate=16000, return_tensors="pt", padding=True)

#logits = model(input_dict.input_values).logits

#pred_ids = torch.argmax(logits, dim=-1)[0]

In [ ]:
#logits.shape

In [ ]:
#" ".join(sorted(processor.tokenizer.get_vocab()))


In [ ]:
#transcription = processor.batch_decode(logits.numpy()).text
#transcription[0].lower()


In [ ]:
#print("Prediction:")
#print(processor.decode(pred_ids))

#print(processor.decode(lingala_test["input_values"][0]))

#print("\nReference:")
#print(common_voice_test_transcription["sentence"][0].lower())
#print(processor.decode(lingala_test["labels"][50]))
#print("Target text:", lingala_test[0]["target_text"])

## test on multiple files

In [29]:
from tqdm import tqdm

In [30]:
final_pred = []
for i in tqdm(range(fleurs_asr_test.shape[0])):    
    input_dict = processor(fleurs_asr_test["input_values"][i],sampling_rate=16000, return_tensors="pt", padding=True)
    with torch.no_grad():
        logits = model(input_dict.input_values).logits

    #pred_ids = torch.argmax(logits, dim=-1)[0]
    #prediction = processor.decode(pred_ids)
    prediction = processor.batch_decode(logits.numpy()).text
    final_pred.append(prediction)

100%|██████████| 209/209 [1:44:08<00:00, 29.90s/it]


In [31]:
final_pred

[['bituka mingi bato bakengaki kuna na babisi ya mike ya japon oyo ezalaka kitoko mpe minene'],
 ['nsima ya ntango bato ya sika babandaki komesana na esika na bango ya sika bakomaki kokesena mingi na bato mosusu'],
 ['ntango akendaki na mboka bapaya mbala liboso bato bazalaki motema molai mpe komitia na esika na yo bayebi ete bato baye na mboka ya sika basengeli komisa'],
 ['ekomaki kosala na mingi kasi ebende ebebisaka lisusu mingi bapno ya mabaya ya bawagon'],
 ['esengaka katikati ya ngonga moko mpona kotambola na mboka kitoko'],
 ['koloba maloba ezalaka mwa pete na lokota ya itali mpo batangaki maloba mingi kaka ndenge bakomiango'],
 ['na ebandeli e lamba ezalaki kolanda mingi izali ya bato ya issatiya este'],
 ['baarabe bamamaki mpe likambo ya mizila na mikili wana pe bana mingi ya komera comorès mpe ya mayeti'],
 ['mbala mingi balobelaka makambo nyonso a ya bozindo mpenza mingimingi balobelaka makambo basi bakutana na yango'],
 ['bato ya siansi bakanisaki ete baochelo balandaka mp

In [32]:
def map_to_pred(batch):
    with torch.no_grad():
      input_values = torch.tensor(batch["input_values"]).unsqueeze(0)
      logits = model(input_values).logits
    #pred_ids = torch.argmax(logits, dim=-1)
    #batch["pred_str"] = processor.batch_decode(pred_ids)[0]
    batch["pred_str"]=processor.batch_decode(logits.numpy()).text[0]
    batch["text"] = processor1.decode(batch["labels"], group_tokens=False)
    return batch

In [ ]:
def map_to_pred(batch):
    with torch.no_grad():
      input_values = torch.tensor(batch["input_values"]).unsqueeze(0)
      logits = model(input_values).logits
    #pred_ids = torch.argmax(logits, dim=-1)
    #batch["pred_str"] = processor.batch_decode(pred_ids)[0]
    batch["pred_str"]=processor.batch_decode(logits.numpy()).text[0]
    batch["text"] = processor.decode(batch["labels"], group_tokens=False)
    return batch

In [33]:
result = fleurs_asr_test.map(map_to_pred, remove_columns=fleurs_asr_test.column_names)

  0%|          | 0/209 [00:00<?, ?ex/s]

In [34]:
result

Dataset({
    features: ['pred_str', 'text'],
    num_rows: 209
})

In [35]:
show_random_elements(result)

,pred_str,text
0,roberta ba azwaki bapoint ki mingi na tour ya liboso ba a se na babala 41 pamba ebetaki 114 na 26,robin uthappa azwaki ba point eleki mingi na toure ya liboso ba point 70 na ba ballon 41 pamba ebetaki 11 quatre na 2 six
1,bandakisa ya misala oyo esalemaka ezali kosala bokila koloba mbisi kokanga bafoto kotala bandeke mpe kotala ba parke mpe koyekola basango etali ndenge mabele ezali,bandakisa ya misala oyo esalemaka ezali kosala bokila koloba mbisi kokanga bafoto kotala bandeke pe kotala baparke mpe koyekola basango etali ndenge mabele ezali
2,noel eza moko ya bafeti ya ntina mingi na boklisto pe bato balobaka ezali mbotama ya eso,noel eza moko ya bafeti ya ntina mingi na boklisto pe bato balobaka ezali mbotama ya yesu
3,mbongwana to bosengi esengeli kosalema na nzela ya kompani ya mobembo kasi te na nzela ya hotel,mbongwana to bosengi nyonso esengeli kosalema na nzela ya kompani ya mobembo kasi te na nzela ya hotel
4,biloko wana yango moko ekoma mateya mpe etiaki likebi mingi na koluka kobongisa makambo bato bakutanaka na yango na bomoi,biloko wana yango moko ekoma mateya mpe etiaki likebi mingi na koluka kobongisa makambo bato bakutanaka na yango na bomoi
5,bazala ya mai ekoki kozala likanisi malamu mpona kobongisa kokonge,banzela ya mai ekoki kozala likanisi malamu mpona kobongisa conge
6,ba na ye ya mu ezalaki likoko mingi musala kitoko ya bakonzi ya suède esalemaki na dakitcloter newstral na 2000 oyo bakotisa na mokanda ya guinnesse ya misala malamu koleka na mokili,timbre na ye ya 1 000 ezalaki kitoko mingi misala kitoko ya bakonzi ya suéde esalemaki na david klöcker ehrenstrahl na 2000 oyo bakotisa na mokanda ya guinness ya misala malamu koleka na mokili
7,bato bakoki koloba ete biloko basala na maboko eza y kala kasi eza ya sika soki tokokisaeni yango na biloko ya ndenge moko oyo esalemi ebele,bato bakoki koloba ete biloko basala na maboko eza ya kala kasi eza ya sika soki tokokanisi yango na biloko ya ndenge moko oyo esalemi ebele
8,part ezali 19 500 km cas pe ekabwani na biteni 14 ekeke eteni moko moko esungaka banyama na ndenge na ndenge,parke ezali 19 500 km pe ekabwani na biteni 14 ekesene eteni mokomoko esungaka banyama ya ndenge na ndenge
9,tuasiansi balobi ete mosala enyama wana ezalaki motali makasi na likolo se na yango ezalaki ya kozuluda to ma langi mingi,bato ya siansi balobi ete masala ya nyama wana ezalaki motane makasi na likolo se na yango ezalaki ya konzuluka to na langi mingi


In [36]:
from datasets import load_dataset, load_metric

In [37]:
#def compute_metrics(pred):
#    pred_logits = pred.predictions
#    pred_ids = np.argmax(pred_logits, axis=-1)
#
#    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id
#
#    pred_str = processor.batch_decode(pred_logits.numpy())
    # we do not want to group tokens when computing the metrics
#    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

#    wer = wer_metric.compute(predictions=pred_str, references=label_str)

#    return {"wer": wer}

In [38]:
from jiwer import wer

In [39]:
print("Test WER: {:.3f}".format(wer(result["pred_str"], result["text"])))

Test WER: 0.211
